In [1]:
##Author: Wong Yuk Ming, Edward
##Remarks: it seems using keras does not require us to convert numpy array to tensor, it will done once we fit the data
#References:
#https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/
import os 
import numpy as np
import tensorflow as tf
from numpy import load

#loading the file from desire directory
path = "../itasserSx9" ##<-modify this directory is ok

filenames= os.listdir(path)
x_List=[]
y_List=[]
xyfileTuple=[]

#since on linux file are not ordered by name, there is no way to guarentee n+1 file is the y file of x file n
#there will not be any selection that limit the number of file to process

for names in filenames:
    if names.endswith("-x-.npy"):
        x_List.append(names)
    else:
        y_List.append(names)

##testing purpose
#print(x_List)
#print(y_List)

for xfile in x_List:
    for yfile in y_List:
        if (xfile.replace('-x-.npy','') == yfile.replace('-y-.npy','')):
            xyfileTuple.append((xfile,yfile))
            break
##testing purpose
#print(xyfileTuple)

#notice that this cell has no error
print('complete without error')

C:\Users\Edward\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Edward\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Edward\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Edward\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

complete without error


In [2]:
#from filename convert to numpy array 
##input:  array of tuple contains x file name and y file name, purpose <"angle", "nonangle">
##output: array of tuple contains x numpy array and y numpy array
def extractData(tupleToConv, purpose):
    outputTensorTuple=[]
    i=0
    for pair in tupleToConv:
        if(purpose=='angle'):
            xdatanp=load(path+"/"+pair[0])
            #xdatatf=tf.convert_to_tensor(xdatanp, dtype=np.float32)
            ydatanp=load(path+"/"+pair[1])
            #print(ydatanp)
            ydatanp=ydatanp[:,4:6]
            #if(i==0):
                #print('from y')
                #print(ydatanp)
            #ydatatf=tf.convert_to_tensor(xdatanp, dtype=np.float32)
            outputTensorTuple.append((xdatanp,ydatanp))
            
        else:
            xdatanp=load(path+"/"+pair[0])
            #xdatatf=tf.convert_to_tensor(xdatanp, dtype=np.float32)
            
            ydatanp=load(path+"/"+pair[1])
            ydatanp=ydatanp[:,:4]
            if(i==0):
                print(ydatanp)
            #ydatatf=tf.convert_to_tensor(xdatanp, dtype=np.float32)
            outputTensorTuple.append((xdatanp,ydatanp))
        
        i=i+1
            
    return outputTensorTuple

angleAllData = extractData(xyfileTuple,purpose='angle')
ssAllData = extractData(xyfileTuple,purpose='nonangle')

#notice that this cell has no error
print('complete without error')

[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
complete without error


In [3]:
#extract to k-mers where k=18 for each amino acid position
#for amino acid at position n, x= n-18:n, y=ny
#also add padding


angleKmersData=[]
ssKmersData=[]

#specify how many file want to process in here
numFileProcess=100 ##<-modify this for changing -1 = process all file
if (numFileProcess==-1):
    numFilProcess=len(angleAllData)

for i in range(numFileProcess):
    thisAngleTuple = angleAllData[i]
    thisSSTuple = ssAllData[i]
    #orgAngleLenght=thisAngleTuple[0].shape[0]
    #orgSSLenght=thisSSTuple[0].shape[0]
    thisAngleTuple=(np.insert(thisAngleTuple[0],0,np.zeros((17,9)),axis=0),thisAngleTuple[1])
    thisSSTuple=(np.insert(thisSSTuple[0],0,np.zeros((17,9)),axis=0),thisSSTuple[1])
    thisAngleTuple=(np.append(thisAngleTuple[0],np.zeros((17,9)),axis=0),thisAngleTuple[1])
    thisSSTuple=(np.append(thisSSTuple[0],np.zeros((17,9)),axis=0),thisSSTuple[1])
    for j in range(thisAngleTuple[1].shape[0]):
        x_vec=None
        y_vec=None
        x_vec=thisAngleTuple[0][j:j+35,:]
        y_vec=thisAngleTuple[1][j,:]
        angleKmersData.append((x_vec,y_vec))

    for j in range(thisSSTuple[1].shape[0]):
        x_vec=None
        y_vec=None    
        x_vec=thisSSTuple[0][j:j+35,:]
        y_vec=thisSSTuple[1][j,:]
        ssKmersData.append((x_vec,y_vec))
       
'''
for i in range(numFileProcess):
    thisAngleTuple = angleAllData[i]
    thisSSTuple = ssAllData[i]
    ##reporter
    #if(i==0):
        #print(thisAngleTuple[0].shape)
        #print(thisAngleTuple[0][0:18,:])
    for j in range(thisAngleTuple[0].shape[0]):
        x_vec=None
        y_vec=None
        if(j<18):
            #print(j)
            x_vec=thisAngleTuple[0][0:j+1,:]
            x_vec=np.insert(x_vec, 0 , np.zeros([17-j,9]) , axis=0)#adding padding
            #print(x_vec)
            #print(x_vec.shape)
        else:
            #print(j)
            x_vec=thisAngleTuple[0][j-17:j+1,:]
            #if(j==18):
            #    print(x_vec)
            #print(x_vec.shape)
        y_vec=thisAngleTuple[1][j,:]
        #x_vec=tf.convert_to_tensor(x_vec, dtype=np.float32)
        #y_vec=tf.convert_to_tensor(y_vec, dtype=np.float32)
        angleKmersData.append((x_vec,y_vec))
    for j in range(thisAngleTuple[0].shape[0]):
        x_vec=None
        y_vec=None
        if(j<18):
            #print(j)
            x_vec=thisSSTuple[0][0:j+1,:]
            x_vec=np.insert(x_vec, 0 , np.zeros([17-j,9]) , axis=0)#adding padding
            #print(x_vec)
            #print(x_vec.shape)
        else:
            #print(j)
            x_vec=thisSSTuple[0][j-17:j+1,:]
            #if(j==18):
            #    print(x_vec)
            #print(x_vec.shape)
        y_vec=thisSSTuple[1][j,:]
        #x_vec=tf.convert_to_tensor(x_vec, dtype=np.float32)
        #y_vec=tf.convert_to_tensor(y_vec, dtype=np.float32)
        ssKmersData.append((x_vec,y_vec))
'''
'''
for tup in angleKmersData:
    print(str(tup[0].shape)+" = "+str(tup[1].shape))
for tup in ssKmersData:
    print(str(tup[0].shape)+" = "+str(tup[1].shape))
'''
#notice that this cell has no error
print('complete without error')

complete without error


In [4]:
#for feeding into the model, it has to be converted from list of 2D array to a 3D array
#for angle
from math import floor
from random import random
Xa=list()
Ya=list()

Xatest=list()
Yatest=list()
#for secondary structure
Xs=list()
Ys=list()

Xstest=list()
Ystest=list()
for item in angleKmersData:
    splitting=floor(random()*10)
    if(splitting>7):
        Xatest.append(item[0])
        Yatest.append(item[1])
    else:
        Xa.append(item[0])
        Ya.append(item[1])
for item in ssKmersData:
    splitting=floor(random()*10)
    if(splitting>7):
        Xstest.append(item[0])
        Ystest.append(item[1])
    else:
        Xs.append(item[0])
        Ys.append(item[1])

# checking
#for i in range(len(Xa)):
#	print(Xa[i], Ya[i])
Xa=np.array(Xa) #[samples=S timestamp=18 features=9]
Ya=np.array(Ya) #[sample=S features=2]
Xs=np.array(Xs)
Ys=np.array(Ys)

Xatest=np.array(Xatest) #[samples=S timestamp=18 features=9]
Yatest=np.array(Yatest) #[sample=S features=2]
Xstest=np.array(Xstest)
Ystest=np.array(Ystest)

#print(Xa.shape)
#Xa=Xa.reshape((Xa.shape[0],Xa.shape[1],9))
print(Ya.shape)
print(Xa.shape)

#notice that this cell has no error
print('complete without error')

(42140, 2)
(42140, 35, 9)
complete without error


In [5]:
#import all things needed for tensorflow LSTM network
#this is for predicting the next number
from tensorflow.keras.models import Sequential #Sequential model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Flatten #CNN
from tensorflow.keras.layers import TimeDistributed #CNN
from tensorflow.keras.layers import Conv1D       #CNN
from tensorflow.keras.layers import MaxPooling1D #CNN

#notice that this cell has no error
print('complete without error')

complete without error


In [6]:
##setting up callback for tensor board
# Load the TensorBoard notebook extension

# Clear any logs from previous runs

import datetime
#log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

##saving checkpoint
model_checkpoint_callback1 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./saved_model/checkpoint/max',
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True
    
)
model_checkpoint_callback2 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./saved_model/checkpoint/e50',
    save_weights_only=False,
    save_freq=50,
    save_best_only=False,
)

##setting up model

# for CNN-LSTM reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
# if not using CNN, just comment this line
#Xa=Xa.reshape(Xa.shape[0],Xa.shape[1],Xa.shape[2],1)

#model
model = Sequential()
#model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, 9,1)))
#model.add(TimeDistributed(MaxPooling1D(pool_size=9)))
#model.add(TimeDistributed(Flatten()))
model.add(Bidirectional(LSTM(100, activation='tanh',return_sequences=True, recurrent_activation='sigmoid', use_bias=True , input_shape=(35,9))))
#model.add(Bidirectional(LSTM(200, activation='relu',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
model.add(Bidirectional(LSTM(100, activation='tanh',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
#model.add(Bidirectional(LSTM(150, activation='tanh',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
#model.add(Bidirectional(LSTM(256, activation='relu',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
#model.add(Bidirectional(LSTM(256, activation='tanh',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
#model.add(Bidirectional(LSTM(256, activation='relu',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
#model.add(Bidirectional(LSTM(100, activation='tanh',return_sequences=True, recurrent_activation='sigmoid', use_bias=True )))
model.add(Bidirectional(LSTM(100, activation='tanh',  recurrent_activation='sigmoid', use_bias=True )))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#fitting the model
model.fit(x=Xs,
          y=Ys,
          batch_size=1000,
          epochs=100,
          verbose=1,
          callbacks=[#tensorboard_callback,
              model_checkpoint_callback1,
              model_checkpoint_callback2 ],
          validation_data=(Xstest,Ystest)
         )
#starting tensorboard

model.save('./saved_model/ssModel')

#notice that this cell has no error
print('complete without error')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 42145 samples, validate on 10449 samples
Epoch 1/100
42145/42145 [==============================] - 17s 399us/sample - loss: 1.3730 - acc: 0.3721 - val_loss: 1.3620 - val_acc: 0.3936
Epoch 2/100
42145/42145 [==============================] - 14s 337us/sample - loss: 1.3175 - acc: 0.3891 - val_loss: 1.2939 - val_acc: 0.4091
Epoch 3/100
42145/42145 [==============================] - 14s 334us/sample - loss: 1.2865 - acc: 0.4102 - val_loss: 1.2773 - val_acc: 0.4265
Epoch 4/100
42145/42145 [===========================

42145/42145 [==============================] - 14s 338us/sample - loss: 0.3278 - acc: 0.8741 - val_loss: 0.9607 - val_acc: 0.6726
Epoch 49/100
42145/42145 [==============================] - 14s 336us/sample - loss: 0.3167 - acc: 0.8738 - val_loss: 1.0664 - val_acc: 0.6385
Epoch 50/100
42145/42145 [==============================] - 14s 342us/sample - loss: 0.2999 - acc: 0.8843 - val_loss: 1.0947 - val_acc: 0.6493
Epoch 51/100
42145/42145 [==============================] - 14s 337us/sample - loss: 0.2967 - acc: 0.8826 - val_loss: 1.0410 - val_acc: 0.6626
Epoch 52/100
42145/42145 [==============================] - 14s 343us/sample - loss: 0.2859 - acc: 0.8894 - val_loss: 1.0239 - val_acc: 0.6590
Epoch 53/100
42145/42145 [==============================] - 14s 336us/sample - loss: 0.2682 - acc: 0.8973 - val_loss: 1.1134 - val_acc: 0.6592
Epoch 54/100
42145/42145 [==============================] - 15s 352us/sample - loss: 0.2607 - acc: 0.8994 - val_loss: 1.0717 - val_acc: 0.6703
Epoch 55/100

In [7]:
### print('training done')
#demonstrate prediction
x_input = Xa[0,:,:]
x_input=x_input.reshape((1,35,9))
yhat = model.predict(x_input,verbose=1)
print(yhat)


1/1 [==============================] - 0s 292ms/sample
[[4.1650012e-05 3.0309115e-09 2.7856034e-10 9.9995840e-01]]
